#### Import libraries

In [1]:
import time
import pickle
import numpy as np
import plotly.express as px
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

#### Initialize parameters

In [28]:
n = 10000 # number of samples
d = 100 # number of dimensions
num_clusters = 5 # number of clusters
cutoff = 5000 # number of samples to use for the initial clustering

filtered_num_clusters = num_clusters-2 # classes to use for initial clustering

#### Generate data with all clusters

In [29]:
X, real_clusters = make_blobs(
    n_samples=n, 
    n_features=d, 
    centers=num_clusters, 
    cluster_std=3, 
    shuffle=True, 
    random_state=42
    )

#### Filter initial clustering data (train) to selected clusters

In [30]:
real_train_clusters = real_clusters[:cutoff]
X_train = X[:cutoff]
X_test = X[cutoff:]

X_train = X_train[np.isin(real_train_clusters, range(filtered_num_clusters))]
real_train_clusters = real_train_clusters[np.isin(real_train_clusters, range(filtered_num_clusters))]

This results in X_train having only 3 clusters, while X_test keeps all 5 clusters.
This way X_test will have to introduce new clusters.

#### Agglomerative clustering

In [31]:
Z = linkage(X_train, 'ward')
train_clusters = fcluster(Z, filtered_num_clusters, criterion='maxclust')

#### Training KNN classifier on the cluster data to use for inference of test data

In [32]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, train_clusters)
test_clusters = knn.predict(X_test)

confusion_matrix(real_clusters[cutoff:], test_clusters)

array([[   0,    0,    0, 1005,    0],
       [   0,  978,    0,    0,    0],
       [   0,    0, 1029,    0,    0],
       [   0,  975,    5,    4,    0],
       [   0,    0,    4, 1000,    0]])

As the confusion matrix shows, the predicted classes overlap, and the 2 clusters that were not present in the training data are not predicted at all.

##### Calculate distance between test data and clusters

In [33]:
dists = knn.kneighbors(X_test, return_distance=True)[0].flatten()
dists.shape

(5000,)

#### Histogram of distances

In [34]:
fig = px.histogram(
    x=dists,
    color=real_clusters[cutoff:],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    barmode='stack',
    labels={'x':'Distance', 'color':'True cluster', 'value':'Count'},
    nbins=100,
    title='Distribution of distances for each cluster'
    )
fig.show()

As the histogram shows, the distances have normal distribution, but a very long right tail. We can't use outlier detection to separate the points that need new clusters. The threshold neither can be set to a fixed value, as it depends on the data and its dimensions, nor will it provide a perfect separation every time.

### Iterative thresholding of distances to find cutoff for new clusters

In [91]:
# calculate threshold based on silhouette score
score_dict = {}
for threshold in np.histogram(dists, bins=20)[1]:
    X_test_to_classify = X_test[dists <= threshold]
    test_to_classify_clusters = knn.predict(X_test_to_classify)
    X_test_to_cluster = X_test[dists > threshold]
    Z = linkage(X_test_to_cluster, 'ward')
    test_to_cluster_clusters = fcluster(Z, classes-len(filtered_classes), criterion='maxclust')
    test_clusters = 
    score = silhouette_score(X_test, test_clusters)
    score_dict[threshold] = score